## deploy

In [1]:
import pandas as pd
import numpy as np
import re
import os

In [2]:
path_dir = r'C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small'
src_dir = os.path.join(path_dir,'txt files')
dst_tmp_dir = os.path.join(path_dir,'csv files temp')
dst_dir = os.path.join(path_dir,'csv files')

lst_files = os.listdir(src_dir)

In [3]:
iter_tp = 0
for iter_fl in lst_files:
    print(iter_tp,iter_fl)
    iter_tp+=1

0 personalized-dialog-task1-API-calls-dev.txt
1 personalized-dialog-task1-API-calls-trn.txt
2 personalized-dialog-task1-API-calls-tst-OOV.txt
3 personalized-dialog-task1-API-calls-tst.txt
4 personalized-dialog-task2-API-refine-dev.txt
5 personalized-dialog-task2-API-refine-trn.txt
6 personalized-dialog-task2-API-refine-tst-OOV.txt
7 personalized-dialog-task2-API-refine-tst.txt
8 personalized-dialog-task3-options-dev.txt
9 personalized-dialog-task3-options-trn.txt
10 personalized-dialog-task3-options-tst-OOV.txt
11 personalized-dialog-task3-options-tst.txt
12 personalized-dialog-task4-info-dev.txt
13 personalized-dialog-task4-info-trn.txt
14 personalized-dialog-task4-info-tst-OOV.txt
15 personalized-dialog-task4-info-tst.txt
16 personalized-dialog-task5-full-dialogs-dev.txt
17 personalized-dialog-task5-full-dialogs-trn.txt
18 personalized-dialog-task5-full-dialogs-tst-OOV.txt
19 personalized-dialog-task5-full-dialogs-tst.txt


## task 1,2

In [5]:
fl_no = 10
path_final = os.path.join(src_dir,lst_files[fl_no])
print(path_final)

C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\full\txt files\personalized-dialog-task3-options-tst-OOV.txt


In [50]:
df = pd.DataFrame([])
path_to_use = path_final
ct = 0
with open(path_to_use) as topo_file:
    cnt = 0
    cnt_sub = 0
    ar_3,ar_4 = [],[]
    ar_1,ar_2 = [],[]
    ar_6 = []


    for line in topo_file:
        #if ct>200: break
        ct+=1

        line = line.strip()    
        if not line.strip(): continue

        init_num = re.findall(r'\d+',line)        

        if init_num[0]=='1':

            #print(line)
            line_splits = line[1:].split()
            ar_1.append(line_splits[0])
            ar_2.append(line_splits[1])
            ar_5 = cnt
            cnt+=1

            if len(ar_3)==0: continue
            #print(ar_6)
            ar_6 = np.unique(np.array(ar_6)).tolist()
            ar_6 = " ".join(ar_6)
            ar_6 = '<KB> ' + ar_6 
            ar_6 = [ar_6]*(len(ar_3)-1)
            ar_6.insert(0,'')
            #print(ar_6)
            #print(np.unique(np.array(ar_6)))
            
            data = list(
                zip(
                    [ar_1[0]]*len(ar_3),
                    [ar_2[0]]*len(ar_3),                    
                    ar_3,
                    ar_4,
                    [ar_5]*len(ar_3),
                    ar_6       
                    ))

            df = df.append(data,ignore_index=True)
            cnt_sub = 0
            ar_3,ar_4,ar_6 = [],[],[]
            ar_1.pop(0)
            ar_2.pop(0)
            continue

        line_splits = line[len(str(init_num[0]))+1:].split('\t')
        # print(line)
        # print(len(str(init_num[0])))
        # print(line[len(str(init_num[0]))+1:])
        # print(line_splits)
        if len(line_splits)!=2:
            line_splits = line[len(str(init_num[0]))+1:].split(' ')            
            line_splits = list(filter(None, line_splits))
            #print(line_splits)
            ar_6.append(line_splits[0])
            #print(line)
            #print(line_splits[0])
            continue   
        ar_3.append(line_splits[0])
        ar_4.append(line_splits[1])
        cnt_sub+=1

df = df.rename(columns={
     0: 'Gender',
     1: 'Age',
     2: 'Call',
     3: 'Response',
     4: 'Count',
     5: 'KB'
     })
print(df.shape)
fl_dst = lst_files[fl_no][:-3] + 'csv'
path_dst_tmp = os.path.join(dst_tmp_dir,fl_dst)
df.to_csv(path_dst_tmp,index=False)

(142831, 6)


In [52]:
df.head()

,Gender,Age,Call,Response,Count,KB
0,female,young,hello,hey girl how is it going,1,
1,female,young,can you book a table in seoul in a moderate pr...,be right back with your reservation,1,<KB> resto_seoul_moderate_korean_1stars_1 re...
2,female,young,<SILENCE>,sure finding some options,1,<KB> resto_seoul_moderate_korean_1stars_1 re...
3,female,young,<SILENCE>,how about this one: resto_seoul_moderate_korea...,1,<KB> resto_seoul_moderate_korean_1stars_1 re...
4,female,young,i love that,awesome you are done,1,<KB> resto_seoul_moderate_korean_1stars_1 re...


In [53]:
TAGS = [

    '<profile> ',
    ' <context>',
    ' <user> ',
    ' <system> ',
    ' <query> ']

c = 0
mn_ar_1 = []
mn_ar_2 = []
mn_ar_3 = []
prev_count = 0
prev_string = ''
for i in range(len(df)):    
    #if c > 100: break
    c+=1    

    if df.iloc[i][4]!=prev_count:
        prev_string = ''
        prev_count = df.iloc[i][4]
    
    a2 = df.iloc[i][2]
    a3 = df.iloc[i][3]    
    call_string = prev_string + TAGS[4]+ a2    
    prev_string = prev_string + TAGS[2]+  a2 + TAGS[3] + a3
    # print(a2)
    # print('call: ',call_string)
    # print('prev: ',call_string)
    # print(a3)
    to_append = TAGS[0]+ line
    mn_ar_1.append(call_string)
    mn_ar_2.append(a3)

ndf = pd.DataFrame(            
            list(zip(
                mn_ar_1,mn_ar_2                  
                )),

            columns =[
                'Call',
                'Response'
                ]
                ) 

df['Profile'] = df['Gender'] + ' ' +df['Age']
ndf['Call'] = TAGS[0]+ df['Profile'] + df['KB'] + TAGS[1] + ndf['Call']

print(ndf.shape)
fl_dst = lst_files[fl_no][:-3] + '.csv'
path_dst = os.path.join(dst_dir,fl_dst)
ndf.to_csv(path_dst,index=False)
    

(142831, 2)


In [60]:
ndf.head()

,Call,Response
0,<profile> female young <context> <query> hello,hey girl how is it going
1,<profile> female young <KB> resto_seoul_moder...,be right back with your reservation
2,<profile> female young <KB> resto_seoul_moder...,sure finding some options
3,<profile> female young <KB> resto_seoul_moder...,how about this one: resto_seoul_moderate_korea...
4,<profile> female young <KB> resto_seoul_moder...,awesome you are done


In [4]:
def convert_to_df(path_final):
    df = pd.DataFrame([])
    path_to_use = path_final
    ct = 0
    with open(path_to_use) as topo_file:
        cnt = 0
        cnt_sub = 0
        ar_3,ar_4 = [],[]
        ar_1,ar_2 = [],[]
        ar_6 = []


        for line in topo_file:
            #if ct>200: break
            ct+=1

            line = line.strip()    
            if not line.strip(): continue

            init_num = re.findall(r'\d+',line)        

            if init_num[0]=='1':

                #print(line)
                line_splits = line[1:].split()
                ar_1.append(line_splits[0])
                ar_2.append(line_splits[1])
                ar_5 = cnt
                cnt+=1

                if len(ar_3)==0: continue
                #print(ar_6)
                ar_6 = np.unique(np.array(ar_6)).tolist()
                ar_6 = " ".join(ar_6)
                ar_6 = ' <KB> ' + ar_6 
                ar_6 = [ar_6]*(len(ar_3)-1)
                ar_6.insert(0,'')
                #print(ar_6)
                #print(np.unique(np.array(ar_6)))
                
                data = list(
                    zip(
                        [ar_1[0]]*len(ar_3),
                        [ar_2[0]]*len(ar_3),                    
                        ar_3,
                        ar_4,
                        [ar_5]*len(ar_3),
                        ar_6       
                        ))

                df = df.append(data,ignore_index=True)
                cnt_sub = 0
                ar_3,ar_4,ar_6 = [],[],[]
                ar_1.pop(0)
                ar_2.pop(0)
                continue

            line_splits = line[len(str(init_num[0]))+1:].split('\t')
            # print(line)
            # print(len(str(init_num[0])))
            # print(line[len(str(init_num[0]))+1:])
            # print(line_splits)
            if len(line_splits)!=2:
                line_splits = line[len(str(init_num[0]))+1:].split(' ')            
                line_splits = list(filter(None, line_splits))
                #print(line_splits)
                ar_6.append(line_splits[0])
                #print(line)
                #print(line_splits[0])
                continue   
            ar_3.append(line_splits[0])
            ar_4.append(line_splits[1])
            cnt_sub+=1

    df = df.rename(columns={
        0: 'Gender',
        1: 'Age',
        2: 'Call',
        3: 'Response',
        4: 'Count',
        5: 'KB'
        })
    print(df.shape)
    fl_dst = lst_files[fl_no][:-3] + 'csv'
    path_dst_tmp = os.path.join(dst_tmp_dir,fl_dst)
    df.to_csv(path_dst_tmp,index=False)
    return df

TAGS = [

    '<profile> ',
    ' <context>',
    ' <user> ',
    ' <system> ',
    ' <query> ']
    
def add_context(df):
    prev_count = 0
    txt = []
    for i in range(len(df)):    
        if df.iloc[i][4]!=prev_count:
            prev_count = df.iloc[i][4]
            txt.append('')
            continue
        txt.append(' <context>')  
    df['TAG'] = txt
    return df


def parse_from_df(df):
    c = 0
    mn_ar_1 = []
    mn_ar_2 = []
    mn_ar_3 = []
    prev_count = 0
    prev_string = ''
    for i in range(len(df)):    
        #if c > 100: break
        c+=1    

        if df.iloc[i][4]!=prev_count:
            prev_string = ''
            prev_count = df.iloc[i][4]
        
        a2 = df.iloc[i][2]
        a3 = df.iloc[i][3]    
        call_string = prev_string + TAGS[4]+ a2    
        prev_string = prev_string + TAGS[2]+ a2 + TAGS[3] + a3
        # print(a2)
        # print('call: ',call_string)
        # print('prev: ',call_string)
        # print(a3)
        mn_ar_1.append(call_string)
        mn_ar_2.append(a3)

    ndf = pd.DataFrame(            
                list(zip(
                    mn_ar_1,mn_ar_2                  
                    )),

                columns =[
                    'Call',
                    'Response'
                    ]
                    ) 

    df['Profile'] = df['Gender'] + ' ' +df['Age']
    ndf['Call'] = TAGS[0]+ df['Profile'] + df['KB'] + TAGS[1] + ndf['Call']

    print(ndf.shape)
    fl_dst = lst_files[fl_no][:-3] + '.csv'
    path_dst = os.path.join(dst_dir,fl_dst)
    ndf.to_csv(path_dst,index=False)       

In [5]:
for i in range(7,12):
    fl_no = i
    path_final = os.path.join(src_dir,lst_files[fl_no])
    print(i,path_final)

7 C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small\txt files\personalized-dialog-task2-API-refine-tst.txt
8 C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small\txt files\personalized-dialog-task3-options-dev.txt
9 C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small\txt files\personalized-dialog-task3-options-trn.txt
10 C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small\txt files\personalized-dialog-task3-options-tst-OOV.txt
11 C:\Users\ydjoe\Documents\nlp\personalized-dialog-dataset\small\txt files\personalized-dialog-task3-options-tst.txt


In [6]:
for i in range(8,12):
    fl_no = i
    path_final = os.path.join(src_dir,lst_files[fl_no])
    df = convert_to_df(path_final)
    df = add_context(df)
    parse_from_df(df)
    

(11722, 6)
(11722, 2)
(11670, 6)
(11670, 2)
(11595, 6)
(11595, 2)
(11747, 6)
(11747, 2)
